In [10]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (3).csv


In [5]:
import pandas as pd
import io

# Load the CSV into a pandas DataFrame
df = pd.read_csv('data.csv')

# Check the first few rows of the data to ensure it loaded correctly
df.head()

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,P
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,P
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,P
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,P
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,P


In [6]:
# Check for missing values
df.isnull().sum()


,0
ID,0
air_time1,0
disp_index1,0
gmrt_in_air1,0
gmrt_on_paper1,0
...,...
paper_time25,0
pressure_mean25,0
pressure_var25,0
total_time25,0


In [7]:
import pandas as pd
import io
import numpy as np

print(df.dtypes)

# Select only numeric columns for calculating the mean
numeric_df = df.select_dtypes(include=np.number)

# Fill missing values in numeric columns with their respective means
df[numeric_df.columns] = numeric_df.fillna(numeric_df.mean())

ID                  object
air_time1            int64
disp_index1        float64
gmrt_in_air1       float64
gmrt_on_paper1     float64
                    ...   
paper_time25         int64
pressure_mean25    float64
pressure_var25     float64
total_time25         int64
class               object
Length: 452, dtype: object


In [8]:
df = df.drop('ID', axis=1)

In [9]:
df['class'] = df['class'].map({'H': 0, 'P': 1})

In [10]:
# Separate features and target
X = df.drop('class', axis=1)
y = df['class']

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
from sklearn.preprocessing import LabelEncoder
y_encoded = LabelEncoder().fit_transform(y)

# MLP

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)

# Train the MLP model
mlp.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = mlp.predict(X_test)
print(f'MLP Accuracy: {accuracy_score(y_test, y_pred)}')


MLP Accuracy: 0.8571428571428571


# CNN Model (Using 1D Convolution)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam

# Reshape the data to match CNN input (samples, timesteps, features)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the CNN model
cnn = Sequential()
cnn.add(Conv1D(64, 2, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)))
cnn.add(MaxPooling1D(2))
cnn.add(Dropout(0.5))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))  # Use softmax if it's a multi-class classification problem

# Compile the model
cnn.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
cnn.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluate the CNN model
cnn_accuracy = cnn.evaluate(X_test_reshaped, y_test)
print(f'CNN Accuracy: {cnn_accuracy[1]}')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.5690 - loss: 0.7082 - val_accuracy: 0.6571 - val_loss: 0.7859
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7297 - loss: 0.5641 - val_accuracy: 0.8571 - val_loss: 0.3503
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8549 - loss: 0.2885 - val_accuracy: 0.8286 - val_loss: 0.2874
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9828 - loss: 0.1407 - val_accuracy: 0.8571 - val_loss: 0.2721
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9681 - loss: 0.1116 - val_accuracy: 0.9143 - val_loss: 0.2670
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9598 - loss: 0.0780 - val_accuracy: 0.8857 - val_loss: 0.2898
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9963 - loss: 0.0346 - val_accuracy: 0.8857 - val_loss: 0.3307
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.0256 - val_accuracy: 0.9143 - val_loss: 0.3103
Epoch 9/10


# LSTM

In [16]:
from tensorflow.keras.layers import LSTM

# Build the LSTM model
lstm = Sequential()
lstm.add(LSTM(64, return_sequences=True, input_shape=(X_train_reshaped.shape[1], 1)))
lstm.add(LSTM(32))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(1, activation='sigmoid'))  # Use softmax for multi-class

# Compile the model
lstm.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluate the LSTM model
lstm_accuracy = lstm.evaluate(X_test_reshaped, y_test)
print(f'LSTM Accuracy: {lstm_accuracy[1]}')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 514ms/step - accuracy: 0.5084 - loss: 0.6860 - val_accuracy: 0.4286 - val_loss: 0.6811
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 374ms/step - accuracy: 0.5720 - loss: 0.6755 - val_accuracy: 0.4571 - val_loss: 0.6768
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 373ms/step - accuracy: 0.5382 - loss: 0.6751 - val_accuracy: 0.5714 - val_loss: 0.6719
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 373ms/step - accuracy: 0.5725 - loss: 0.6647 - val_accuracy: 0.5429 - val_loss: 0.6677
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 587ms/step - accuracy: 0.5645 - loss: 0.6669 - val_accuracy: 0.4857 - val_loss: 0.6711
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 377ms/step - accuracy: 0.5690 - loss: 0.6592 - val_accuracy: 0.4857 - val_loss: 0.6741
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 373ms/step - accuracy: 0.5764 - loss: 0.6443 - val_accuracy: 0.6000 - val_loss: 0.6590
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 370ms/step - accuracy: 0.5910 - loss: 0.6353 - val_accuracy: 0.6571 - val_loss: 0.6452
Epo

# SVM

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM model
svm = SVC(kernel='linear', random_state=42)

# Train the model
svm.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_svm = svm.predict(X_test)
print(f'SVM Accuracy: {accuracy_score(y_test, y_pred_svm)}')


SVM Accuracy: 0.8857142857142857


# random forest classifier

In [18]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_rf = rf.predict(X_test)
print(f'Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf)}')


Random Forest Accuracy: 0.8857142857142857


# Gradient Boosting Classifer

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting model
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Train the model
gb.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_gb = gb.predict(X_test)
print(f'Gradient Boosting Accuracy: {accuracy_score(y_test, y_pred_gb)}')


Gradient Boosting Accuracy: 0.8857142857142857


# K-nearest neighbor

In [22]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize the KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_knn = knn.predict(X_test)
print(f'KNN Accuracy: {accuracy_score(y_test, y_pred_knn)}')


KNN Accuracy: 0.6


# XG Booster

In [21]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Convert data to DMatrix format, which is required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'binary:logistic',  # Use 'multi:softmax' for multi-class
    'eval_metric': 'error',
    'max_depth': 5,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Train the XGBoost model
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
y_pred_xgb = (xgb_model.predict(dtest) > 0.5).astype(int)  # Threshold at 0.5 for binary classification
print(f'XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb)}')


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:23:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.8857142857142857


# Hyperparameter Tuning

In [23]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the grid search
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and accuracy
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_}")


Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
Best cross-validation accuracy: 0.8920634920634921


# Light GBM

In [31]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Set parameters for LightGBM
params = {
    'objective': 'binary',  # Use 'multiclass' for multi-class classification
    'metric': 'binary_error',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
lgb_model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

# Predict and evaluate the model
y_pred_lgb = (lgb_model.predict(X_test) > 0.5).astype(int)  # Threshold for binary classification
print(f'LightGBM Accuracy: {accuracy_score(y_test, y_pred_lgb)}')



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ke

AttributeError: _ARRAY_API not found

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 69, number of negative: 70
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001631 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19890
[LightGBM] [Info] Number of data points in the train set: 139, number of used features: 450
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.496403 -> initscore=-0.014389
[LightGBM] [Info] Start training from score -0.014389
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

# keras multi-perception

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load your data
# data = pd.read_csv('data.csv')
# X = data.drop('class', axis=1)
# y = data['class']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple MLP model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'MLP Neural Network Accuracy: {test_accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.4605 - loss: 45207.2500 - val_accuracy: 0.4286 - val_loss: 7052.7178
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5003 - loss: 12194.7266 - val_accuracy: 0.5714 - val_loss: 4000.9907
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6185 - loss: 5852.4038 - val_accuracy: 0.7143 - val_loss: 1811.8313
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6722 - loss: 3368.5190 - val_accuracy: 0.7429 - val_loss: 1237.4785
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6990 - loss: 3100.3879 - val_accuracy: 0.6286 - val_loss: 2154.2664
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7303 - loss: 1943.0531 - val_accuracy: 0.7714 - val_loss: 1129.4419
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7948 - loss: 1745.4086 - val_accuracy: 0.8000 - val_loss: 1173.4491
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7955 - loss: 1218.

# voting Classifier

In [27]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

svm_model = SVC(probability=True)
rf_model = RandomForestClassifier(n_estimators=100)
gb_model = GradientBoostingClassifier(n_estimators=100)

voting_model = VotingClassifier(estimators=[
    ('svm', svm_model),
    ('rf', rf_model),
    ('gb', gb_model)
], voting='hard')

voting_model.fit(X_train, y_train)
y_pred_voting = voting_model.predict(X_test)
print(f'Ensemble Voting Accuracy: {accuracy_score(y_test, y_pred_voting)}')



Ensemble Voting Accuracy: 0.8857142857142857


# Result

In [33]:
from sklearn.metrics import classification_report

# Print classification reports for all models
print("MLP Classifier Report:")
print(classification_report(y_test, mlp.predict(X_test)))

print("SVM Classifier Report:")
print(classification_report(y_test, svm.predict(X_test)))

print("Random Forest Classifier Report:")
print(classification_report(y_test, rf.predict(X_test)))

print("Gradient Boosting Classifier Report:")
print(classification_report(y_test, gb.predict(X_test)))

print("KNN Classifier Report:")
print(classification_report(y_test, knn.predict(X_test)))

print("XGBoost Classifier Report:")
print(classification_report(y_test, (xgb_model.predict(dtest) > 0.5).astype(int)))

print("LightGBM Classifier Report:")
print(classification_report(y_test, (lgb_model.predict(X_test) > 0.5).astype(int)))

# Voting Classifier Report
print("Voting Classifier Report:")
print(classification_report(y_test, voting_model.predict(X_test)))


MLP Classifier Report:
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        15
           1       0.94      0.80      0.86        20

    accuracy                           0.86        35
   macro avg       0.86      0.87      0.86        35
weighted avg       0.87      0.86      0.86        35

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.82      0.93      0.88        15
           1       0.94      0.85      0.89        20

    accuracy                           0.89        35
   macro avg       0.88      0.89      0.88        35
weighted avg       0.89      0.89      0.89        35

Random Forest Classifier Report:
              precision    recall  f1-score   support

           0       0.82      0.93      0.88        15
           1       0.94      0.85      0.89        20

    accuracy                           0.89        35
   macro avg       0.88      0.89      0.88      